# Data Understanding

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import folium
import altair as alt
# make your plot outputs appear and be stored within the notebook
%matplotlib inline  

print('Folium installed')
print('Libraries imported.')

In [ ]:
mpl.rcParams['axes.labelsize'] = 12
mpl.rcParams['axes.titlesize'] = 14
mpl.rcParams['figure.titlesize'] = 16

pd.set_option('display.max_columns', 500)

## Import Data

In [ ]:
mps_colp = pd.read_csv('data/new_data/mps_colp.csv')
mps_colp.info()

In [ ]:
mps_colp['Month'] = pd.to_datetime(mps_colp['Month'], format='%Y-%m').dt.strftime('%Y-%m')
mps_colp['Month'].unique()

In [ ]:
mps_colp[['Falls within', 'LSOA code', 'LSOA name', 'Crime type', 'Location', 'Borough']] = \
mps_colp[['Falls within', 'LSOA code', 'LSOA name', 'Crime type', 'Location', 'Borough']].astype((pd.StringDtype()))

In [ ]:
# mps_colp['Month'] = mps_colp['Month'].values.astype('datetime64[M]')

In [ ]:
mps_colp.info(null_counts=True)

In [ ]:
# reshape the dataframe for crime analysis
crime = mps_colp.groupby(['LSOA code', 'LSOA name', 'Borough','Crime type', 'Month']).size().unstack().reset_index(drop=False).fillna(0)
# add Total column to MPS data
crime['Total'] = crime.sum(axis=1, numeric_only=True)
# delete index name
crime.columns.name = None
crime.head(2)

In [ ]:
crime.to_csv('data/new_data/crime_table.csv', index=False)

### BoxPlots

#### Crimes per Borough

<i> Compute the number of crimes per Month and per Borough: </i>

In [ ]:
borough_crimes_all = mps_colp.copy()
borough_crimes_all["Number of Crimes"] = borough_crimes_all.groupby(["Borough", 'Crime type', "Month"])["Month"].transform("size") 
# borough_crimes_all['Month'] = pd.to_datetime(borough_crimes_all['Month'])
borough_crimes_all.head()



<i> Pivot table for number of crimes per Month and Borough</i>

In [ ]:
borough_crimes_pivoted = pd.pivot_table(
    borough_crimes_all, index=["Borough", 'Crime type'], values="Number of Crimes", columns="Month"
).dropna().reset_index()
borough_crimes_pivoted.head(2)

<i> Manipulate data frame to keep only Borough, Number of Crimes and Month </i>

In [ ]:
borough_crimes = pd.melt(borough_crimes_pivoted, id_vars = ["Borough", 'Crime type'], value_name="Number of Crimes")
borough_crimes.head()

In [ ]:
borough_crimes[['Borough', 'Crime type']] = borough_crimes[['Borough','Crime type']].astype((pd.StringDtype()))
borough_crimes.info()

In [ ]:
borough_crimes.to_csv('data/new_data/crimes_by_borough.csv', index=False)

In [ ]:
# disable the max rows that altair has by default
alt.data_transformers.disable_max_rows()

alt.Chart(borough_crimes).mark_boxplot().encode(
    x='Month:T',
    y='Number of Crimes:Q',
)

In [ ]:
alt.Chart(borough_crimes).mark_boxplot().encode(
    x='Number of Crimes:Q',
    y='Borough:N',
)

In [ ]:
borough_crimes.head()

In [ ]:
plt.figure(figsize=(15, 35))

for index, borough in enumerate(borough_crimes['Borough'].unique()):
    plt.subplot(11, 3, index + 1)

    plt.plot(borough_crimes[borough_crimes['Borough'] == borough]['Month'], borough_crimes[borough_crimes['Borough'] == borough]['Number of Crimes'])
    # plt.xticks(ticks=range(9), labels=range(2019, 2021))
    plt.grid()
    plt.title('{}'.format(borough))
    plt.xlabel('Month')
    plt.ylabel('# of Crimes')
    plt.xticks(rotation=90)

plt.suptitle('Crimes per Months', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

In [ ]:
crimes_per_boroughs = pd.crosstab(mps_colp['Month'], mps_colp['Borough'])
plt.figure(figsize=(15,7))
chart = sns.lineplot(
    data=crimes_per_boroughs,
)
plt.xticks(
    rotation=90, 
    horizontalalignment='right',
    fontweight='light',
    fontsize='x-large'  
)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

#### Crime type per month

In [ ]:
# Compute the number of crimes per Month and per Crime type:
crime_month_all = mps_colp.copy()
crime_month_all["Number of Crimes"] = crime_month_all.groupby(["Crime type", "Month"])["Crime type"].transform("size") 
# Pivot table for number of crimes per Month and Crime type:
crimes_month_pivoted = pd.pivot_table(
    crime_month_all, index="Crime type", values="Number of Crimes", columns="Month"
).dropna().reset_index()
# Manipulate data frame to keep only Crime type, Number of Crimes and Month
crimes_by_month = pd.melt(crimes_month_pivoted, id_vars = "Crime type", value_name="Number of Crimes", var_name="Month")
crimes_by_month['Crime type'] = crimes_by_month['Crime type'].astype((pd.StringDtype()))
crimes_by_month.info()

In [ ]:
crimes_by_month.to_csv('data/new_data/crime_by_month.csv', index=False)

In [ ]:
crimes_by_month.head()

In [ ]:
plt.figure(figsize=(15, 15))

for index, crimetype in enumerate(crimes_by_month['Crime type'].unique()):
    plt.subplot(5, 3, index + 1)
    plt.plot(crimes_by_month[crimes_by_month['Crime type'] == crimetype]['Month'], crimes_by_month[crimes_by_month['Crime type'] == crimetype]['Number of Crimes'])
    # plt.xticks(ticks=range(9), labels=range(2019, 2021))
    plt.grid()
    plt.title('{}'.format(crimetype))
    plt.xlabel('Month')
    plt.ylabel('# of Crimes')
    plt.xticks(rotation=90)

plt.suptitle('Crime types per Months', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])



In [ ]:
crime_type_per_month = pd.crosstab(mps_colp['Month'], mps_colp['Crime type'])
plt.figure(figsize=(15,7))
chart = sns.lineplot(
    data=crime_type_per_month,
)
plt.xticks(
    rotation=90, 
    horizontalalignment='right',
    fontweight='light',
    fontsize='x-large'  
)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

# ....

In [ ]:
def proportionCrimeCategory(data, discreteParam):
    """
    Extracts proportion of Crimes in the data for the various values of discreteParam
    :param data: input dataframe
    :param discreteParam: name of column of dataframe 
    :return normedtable: Series
            column 1 : index of month
            column 2 : sum of crimes during this month/total numbers of crimes
    """
    by_param = data.groupby([discreteParam, 'Crime type'])
    #apllying .size() allows to extract number of instances for each Crime type by the offense in each discreteParam
    table = by_param.size()
    #puts it as a 2D table with number of occurences per discreteParam and Crimetype
    d2table = table.unstack()
    #d2table.sum(1) returns the number of crimes in all crimetypes per discreteParam
    #1 is for the number of the axe on which the sum is done. Here :Crimetype
    normedtable = d2table.div(d2table.sum(1), axis=0)
    return normedtable
    print(normedtable)
    
discreteParamList = ['Month', 'Borough']
fig1, axes1 = plt.subplots(len(discreteParamList),1) #creates a 2x1 blank plot
for i in range(len(discreteParamList)): #now we fill in the subplots
    param = discreteParamList[i]
    table = proportionCrimeCategory(borough_crimes_all, param)

    ax = axes1[i]
    #create plot title
    ax.set_title("Categories of crime by %s" % discreteParamList[i])
    ax.title.set_fontsize(20)
    #choose colormap (memo : looks ok : none, prism_r)
    #stacked = true, allows to stack the different categories into one bar
    table.plot(kind='barh', stacked=True, ax=axes1[i],figsize=(30,15), 
               colormap='tab20c')

    #Puts legend only on second graph and pushes the legend out of plot
    if i==1:
        #how to move legends : http://matplotlib.org/examples/pylab_examples/legend_demo3.html
        ax.legend(bbox_to_anchor=(1.12, 2))

    else:
        ax.legend_.remove()
plt.show()
# plt.savefig("input/proportions.pdf",format="pdf")

In [ ]:
import math
mps_colp['crime_count']=1
crime_alt = mps_colp[['Month', 'Longitude', 'Latitude', 'Borough', 'LSOA name','crime_count']].copy()
crime_alt['year'] = pd.DatetimeIndex(crime_alt['Month']).year
crime_alt['month'] = pd.DatetimeIndex(crime_alt['Month']).month


crime_alt['Longitude'] = crime_alt['Longitude'].round(decimals=3)
crime_alt['Latitude'] = crime_alt['Latitude'].round(decimals=3)

# crime_alt["crimeactivity"] = crime_alt.groupby(["year", "month", 'Latitude', 'Longitude'])["crime_count1"].transform("size") 
# crime_alt["crimeactivity"] = crime_alt.groupby(["year", "month", 'Latitude', 'Longitude']).agg('sum')
crime_alt = crime_alt.groupby(["year", "month", 'Latitude', 'Longitude'])["crime_count"].sum()
crime_alt = crime_alt.reset_index(drop=False)
crime_alt = crime_alt.drop_duplicates(["year", "month", 'Latitude', 'Longitude']).reset_index(drop=True)
crime_alt.head()


In [ ]:
# crime_alt.groupby(["Latitude", "Longitude",'year', 'month']).crime_count1.size()

In [ ]:
crime_alt.info()

In [ ]:
from pyproj import Transformer
transformer = Transformer.from_crs("epsg:4326", "EPSG:27700")
crime_alt['x']=crime_alt.apply(lambda row: transformer.transform(row['Latitude'],row['Longitude'])[0],axis=1)
crime_alt['y']=crime_alt.apply(lambda row: transformer.transform(row['Latitude'],row['Longitude'])[1],axis=1)
crime_alt.head(2)

In [ ]:
crime_alt.groupby(['x', 'y']).ngroups

In [ ]:
crime_alt.info()

In [ ]:
alt.data_transformers.disable_max_rows()
alt.Chart(crime_alt).mark_circle().encode(
    #bin into half km squares
    alt.X('x', axis=alt.Axis(labels=False),scale=alt.Scale(zero=False)),
    alt.Y('y', axis=alt.Axis(labels=False),scale=alt.Scale(zero=False)),
    alt.Color('sum(crime_count)',scale=alt.Scale(scheme='purples',domain=[0,20])),
    column='month:Q',
    row='year:Q',
    tooltip=['sum(crime_count)']
).properties(
    width=100,
    height=100
)

In [ ]:
alt.data_transformers.disable_max_rows()
alt.Chart(crime_alt).mark_rect().encode(
    #bin into half km squares
    alt.X('x', bin=alt.Bin(step=1000,nice=True), axis=alt.Axis(labels=False)),
    alt.Y('y', bin=alt.Bin(step=1000,nice=True), axis=alt.Axis(labels=False)),
    alt.Color('sum(crime_count)',scale=alt.Scale(scheme='reds',domain=[0,300])),
    column='month:Q',
    row='year:Q',
    tooltip=['sum(crime_count)']
).properties(
    width=100,
    height=100
)

In [ ]:
mps_colp['Month'] = pd.to_datetime(mps_colp['Month'])

In [ ]:
#import datetime

activityByMonth=pd.DataFrame()
#group by day and hour, summing the activity
activityByMonth['crime_count']=mps_colp.groupby([mps_colp['Month'].dt.year,mps_colp['Month'].dt.month])['crime_count'].sum()

#groupby puts the groups fields into a multilevel index, so we need to extract these and add as columns (and rename, because they are all originally given the name "t")
activityByMonth.reset_index(level=1, inplace=True)
activityByMonth.rename(columns={'Month': 'month'}, inplace=True)
activityByMonth.reset_index(level=0, inplace=True)
activityByMonth.rename(columns={'Month': 'year'}, inplace=True)

#display
activityByMonth.head()

In [ ]:
alt.Chart(activityByMonth).mark_line().encode(
    x=alt.X('month:Q'),
    y = alt.Y('crime_count:Q', scale=alt.Scale(domain=[70000, 120000])),
    # y='crime_count:Q',
    color='year:N',
).properties(
    width=600,
    height=300
)